# MAPS Step 1 - Image Quality Control
After collecting images, follow the steps below to remove out-of-focus images. Artifacts such as foreign objects, air bubbles and over-expressed cells will be eliminated when measuring focus. <br>
Some outputs were saved to show the expected output.

In [1]:
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
from MAPS_util import *

## Set the working directory

In [ ]:
# check current directory
os.getcwd()

In [ ]:
# change directory, if needed
os.chdir('your-directory')
# confirm the directory
os.getcwd()

## Perform image QC
### Note: This program assumes there are 3 channels per field. <br> Therefore, 1 image set has blue, green and red channels.
For example: blue-DAPI, green-GFP, red-TRITC.
Each channel is saved as separate files with suffixes d0, d1 and d2.

In [81]:
folder_path = "source-image-folder" # source

create_folder(folder_path, "not-blurry") # create destination folder, if it doesn't already exist
dst_folder = folder_path + "not-blurry/" # destination

Created folder 'not-blurry'.


In [82]:
# get the file names of the images in the folder
ID_list = get_names(folder_path)
print(f'There are {len(ID_list)} set of images waiting to be processed.')

There are 46 set of images waiting to be processed.


In [83]:
for n, x in enumerate(ID_list):
    blue = None
    green = None
    red = None
    print()
    print(f'Processing image set {n+1} out of {len(ID_list)}. ID={x}')

    # load each channel
    for entry in os.scandir(folder_path): # import each channel
        if x in entry.name and entry.name.endswith('d0.TIF'): # change the suffix of your image files to match
            blue = cv2.imread(entry.path, 0)
            print(f"blue: {entry.name}")
        if x in entry.name and entry.name.endswith('d1.TIF'):
            green = cv2.imread(entry.path, 0)
            print(f"green: {entry.name}")
        if x in entry.name and entry.name.endswith('d2.TIF'):
            red = cv2.imread(entry.path, 0)
            print(f"red: {entry.name}")
    # check if all channels are correctly loaded
    if blue is None:
        print("No blue channel found. Skip...")
        continue
    elif green is None:
        print("No green channel found. Skip...")
        continue
    elif red is None:
        print("No red channel found. Skip...")
        continue
    
    # use DAPI (blue channel) to do the 1st rnd of blurry removal
    edgesBlue = cv2.Canny(blue, 100, 200)
    fmBlue = variance_of_laplacian(edgesBlue)
    
    if fmBlue > 500: # focus threshold, higher = sharper
        print(f'Image might be sharp... 1st focus check FM1={fmBlue:.2f}')
        # blur out over-exposed cells
        mask = find_blobs(red)
        blrred = blur_blobs(red, mask)
                
        # remove air bubbles
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))  # taking a matrix of size 5 as kernel
        morph = cv2.morphologyEx(blrred, cv2.MORPH_OPEN, kernel) # (erosion followed by dilation)
            
        # use TRITC (red channel) to do the 2nd rnd of blurry removal
        edgesRed = cv2.Canny(morph, 50, 200)
        fmRed = variance_of_laplacian(edgesRed)
            
        if fmRed > 200: # focus threshold, higher = sharper
            print(f'Image is sharp! 2nd focus check FM2={fmRed:.2f}')
            # uncomment to perform illumination correction
            #blue = illum_corr(blue)
            green = hist_adj(green, clahe=False, offset=0)
            red = hist_adj(red, clahe=False, offset=0)
                
            mergedImg = cv2.merge([blue, green, red])
            fName = 'merged_' + str(x) + '.TIF'
            print(f"Processed image saved as {fName}", '\n')
            cv2.imwrite(f"{dst_folder}"+fName, mergedImg)
        else:
            print("Image is actually blurry; removed.", '\n')    
    else:
        print("Image is blurry; removed.", '\n')
print("All done!")


Processing image set 1 out of 46. ID=191120110001_C02f45
blue: MFGTMP_191120110001_C02f45d0.TIF
green: MFGTMP_191120110001_C02f45d1.TIF
red: MFGTMP_191120110001_C02f45d2.TIF
Image might be sharp... 1st focus check FM1=4512.85
Image is sharp! 2nd focus check FM2=669.12
Processed image saved as merged_191120110001_C02f45.TIF 


Processing image set 2 out of 46. ID=191120110001_C02f78
blue: MFGTMP_191120110001_C02f78d0.TIF
green: MFGTMP_191120110001_C02f78d1.TIF
red: MFGTMP_191120110001_C02f78d2.TIF
Image might be sharp... 1st focus check FM1=1280.19
Image is actually blurry; removed. 


Processing image set 3 out of 46. ID=191120110001_C02f61
green: MFGTMP_191120110001_C02f61d1.TIF
blue: MFGTMP_191120110001_C02f61d0.TIF
red: MFGTMP_191120110001_C02f61d2.TIF
Image might be sharp... 1st focus check FM1=590.05
Image is actually blurry; removed. 


Processing image set 4 out of 46. ID=191120110001_C02f63
red: MFGTMP_191120110001_C02f63d2.TIF
blue: MFGTMP_191120110001_C02f63d0.TIF
green: MFG

Image is sharp! 2nd focus check FM2=210.60
Processed image saved as merged_191120110001_C02f51.TIF 


Processing image set 33 out of 46. ID=191120110001_C02f48
green: MFGTMP_191120110001_C02f48d1.TIF
blue: MFGTMP_191120110001_C02f48d0.TIF
red: MFGTMP_191120110001_C02f48d2.TIF
Image might be sharp... 1st focus check FM1=1292.22
Image is sharp! 2nd focus check FM2=532.07
Processed image saved as merged_191120110001_C02f48.TIF 


Processing image set 34 out of 46. ID=191120110001_C02f75
green: MFGTMP_191120110001_C02f75d1.TIF
blue: MFGTMP_191120110001_C02f75d0.TIF
red: MFGTMP_191120110001_C02f75d2.TIF
Image might be sharp... 1st focus check FM1=1257.93
Image is sharp! 2nd focus check FM2=413.00
Processed image saved as merged_191120110001_C02f75.TIF 


Processing image set 35 out of 46. ID=191120110001_C02f77
red: MFGTMP_191120110001_C02f77d2.TIF
blue: MFGTMP_191120110001_C02f77d0.TIF
green: MFGTMP_191120110001_C02f77d1.TIF
Image might be sharp... 1st focus check FM1=1867.02
Image is shar